### Install LangChain-Groq Integration

In [71]:
!pip install -qU langchain-groq python-dotenv

### Getting Started With Ollama

[https://itsfoss.com/ollama-setup-linux/](https://itsfoss.com/ollama-setup-linux/)

### Load Envs

In [72]:
from dotenv import load_dotenv

%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [73]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [74]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 55, 'total_tokens': 64, 'completion_time': 0.036, 'prompt_time': 0.01635817, 'queue_time': 0.004929370000000002, 'total_time': 0.05235817}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_b3ae7e594e', 'finish_reason': 'stop', 'logprobs': None}, id='run-484fef8c-188b-41b0-9967-352b93a33a52-0', usage_metadata={'input_tokens': 55, 'output_tokens': 9, 'total_tokens': 64})

In [75]:
ai_msg.content

"J'adore la programmation."

## Question Translation Prompt

In [76]:
!pip install langchain langchain-community

In [77]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
)
from pydantic import BaseModel
from langchain_core.output_parsers import JsonOutputParser


### Creating Parser

In [78]:
class TranslationParser(BaseModel):
    source_language: str
    target_language: str
    target_text: str
    
translation_parser =  JsonOutputParser(pydantic_object=TranslationParser)

### Prompts

In [79]:
language_translation = """
You are a highly skilled assistant specializing in language translation. 
Your task is to accurately translate the given text into English while 
preserving the original meaning, tone, and context. Please translate the user's sentence to English.

User's sentence: {user_sentence}

Your final output should be in the following format: {format_instructions}
"""

In [80]:
language_translation_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["user_sentence", "format_instructions"],
        template=language_translation,
        partial_variables={
            "format_instructions": translation_parser.get_format_instructions()
        },
    )
)

human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["user_sentence"], template="Translate this text to English. Text: {user_sentence}"
    )
)

messages = [language_translation_prompt, human_prompt]

translation_chain_promt = ChatPromptTemplate(
    messages=messages,
    input_variables=["user_sentence"]
)

In [81]:
first_layer_translation_chain = translation_chain_promt | llm | translation_parser

In [82]:
first_layer_translation_chain.invoke({
    "user_sentence": "J'aime programmer."
})

{'source_language': 'French',
 'target_language': 'English',
 'target_text': 'I like programming.'}

## Chatbot

In [83]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from dotenv import load_dotenv
import os

%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


#### Qdrant Client

In [84]:
qdrant_client = QdrantClient(
    api_key=os.getenv("QDRANT_API_KEY"),
    url=os.getenv("QDRANT_URL")
)

In [85]:
COLLECTION_NAME ="healthcare_collection"

In [86]:
from qdrant_client.http.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

In [87]:
embedding_model = OpenAIEmbeddings(
    model=os.getenv("EMBEDDING_MODEL", 
                    default="text-embedding-3-small"
                )
)

In [88]:
vector_store = QdrantVectorStore(
    client=qdrant_client,
    collection_name=COLLECTION_NAME,
    embedding=embedding_model,
    # retrieval_mode=RetrievalMode.DENSE
)

In [89]:
def information_retriever(question: str) -> str:
    """
    Search and retrieve information from the Qdrant vector store.
    """
    
    # Filter the search results based on the product name
    unstructured_data_results = vector_store.similarity_search(
        query=question,
        k=3
    )
    
    return unstructured_data_results

In [90]:
information_retriever("What is malaria?")

[Document(metadata={'Disease': 'Malaria', '_id': '1497e650-06e7-458c-a542-9f8815e7303f', '_collection_name': 'healthcare_collection'}, page_content='Disease: Malaria Description: An infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type. Immediate Action: Consult nearest hospital Medical Advice: avoid oily food Dietary Guidance: avoid non veg food Long-term Management: keep mosquitos out'),
 Document(metadata={'Disease': 'Dengue', '_id': '8320500c-c4d2-4143-9263-83fbd5e4170b', '_collection_name': 'healthcare_collection'}, page_content='Disease: Dengue Description: an acute infectious disease caused by a flavivirus (species Dengue virus of the genus Flavivirus), transmitted by aedes mosquitoes, and characterized by headache, severe joint pain, and a rash. — called also breakbone fever, dengue fever. Immediate Action: dr

### Actual Chatbot

In [91]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
)

In [92]:
bot_chat_template = """
You are an AI healthcare chatbot with extensive knowledge of medical conditions and diseases. Your task is to provide accurate, concise answers to user questions based solely on the given context.

Context: 
{context}

Question: 
{question}

Instructions:
- Respond using natural language, ensuring your answer is clear and to the point.
- After your answer, explicitly state the context used without modification of the information. You are only to modify the datatype.
- If no context is provided, simply reply with: "No context was provided."

Answer:
Context Used:
"""

In [93]:
from langchain_core.documents import Document
from pydantic import Field, BaseModel
from typing import List, Union

In [94]:
class ResponseFormat(BaseModel):
    """Identifying information about Products."""

    answer: str = Field(
        ...,
        description="Your response to the user query."
    )
    sources: List[Union[str| Document]] = Field(
        ...,
        description="The sources, contexts used to generate the response."
    )

In [95]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [96]:
bot_chat_prompt = ChatPromptTemplate.from_template(bot_chat_template)

chat_chain = (
    RunnableParallel(
        {
            "context": information_retriever,
            "question": RunnablePassthrough(),
        }
    )
    | bot_chat_prompt
    | llm.with_structured_output(ResponseFormat)
)  

In [97]:
response = chat_chain.invoke("What is malaria?")

In [98]:
response

ResponseFormat(answer='Malaria is an infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type.', sources=[Document(metadata={'Disease': 'Malaria', '_id': '1497e650-06e7-458c-a542-9f8815e7303f', '_collection_name': 'healthcare_collection'}, page_content='Disease: Malaria Description: An infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type. Immediate Action: Consult nearest hospital Medical Advice: avoid oily food Dietary Guidance: avoid non veg food Long-term Management: keep mosquitos out')])

In [99]:
response.answer

'Malaria is an infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type.'

In [100]:
response.sources

[Document(metadata={'Disease': 'Malaria', '_id': '1497e650-06e7-458c-a542-9f8815e7303f', '_collection_name': 'healthcare_collection'}, page_content='Disease: Malaria Description: An infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type. Immediate Action: Consult nearest hospital Medical Advice: avoid oily food Dietary Guidance: avoid non veg food Long-term Management: keep mosquitos out')]

In [101]:
second_layer_language_translation = """
You are a highly skilled assistant specializing in language translation. 
Your task is to accurately translate the given text into the specified target language while 
preserving the original meaning, tone, and context.

User's sentence: {sentence}
Souce Language: English
Target Language: {target_language}

Your final output should be in the following format: {format_instructions}
"""

In [102]:
second_language_translation_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["sentence", "target_language", "format_instructions"],
        template=second_layer_language_translation,
        partial_variables={
            "format_instructions": translation_parser.get_format_instructions()
        },
    )
)

second_layer_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["sentence"], template="Translate this text to the specified language. Text: {sentence}"
    )
)

messages = [second_language_translation_prompt, second_layer_human_prompt]

second_layer_translation_chain_promt = ChatPromptTemplate(
    messages=messages,
    input_variables=["sentence"]
)

In [103]:
second_layer_translation_chain = second_layer_translation_chain_promt | llm | translation_parser

In [104]:
response = second_layer_translation_chain.invoke({
    "sentence": response.answer,
    "target_language": "French"
})

In [105]:
response

{'source_language': 'English',
 'target_language': 'French',
 'target_text': 'Le paludisme est une maladie infectieuse causée par des parasites protozoaires de la famille du Plasmodium qui peuvent être transmis par la piqûre du moustique Anopheles ou par une aiguille contaminée ou une transfusion. Le paludisme à falciparum est le type le plus mortel.'}

#### Putting It All Together

In [114]:
def multilingual_chatbot(question: str) -> dict:
    """
    Multilingual chatbot that answers user questions in multiple languages.
    """
    # Translate the user question to English
    translated_question = first_layer_translation_chain.invoke({
        "user_sentence": question
    })
    
    # Retrieve information based on the user question
    response = chat_chain.invoke(translated_question.get("target_text"))
    
    # Translate the response to the specified target language
    translated_response = second_layer_translation_chain.invoke({
        "sentence": response.answer,
        "target_language": translated_question.get("source_language")
    })
    
    return {
        "question": translated_question, 
        "response": translated_response, 
        "sources": response.sources
    }

In [115]:
ml_response = multilingual_chatbot("Malaria ni nini?")

In [116]:
ml_response

{'question': {'source_language': 'Swahili',
  'target_language': 'English',
  'target_text': 'What is malaria?'},
 'response': {'source_language': 'English',
  'target_language': 'Swahili',
  'target_text': "Malaria ni ugonjwa wa kuambukiza unaosababishwa na vimelea vya protozoa kutoka kwa familia ya Plasmodium ambavyo vinaweza kusambazwa kwa kung'ata kwa mbu wa Anopheles au kwa kuchoma sindano au kwa njia ya damu. Malaria ya Falciparum ni aina inayoweza kuua zaidi."},
 'sources': [Document(metadata={'Disease': 'Malaria', '_id': '1497e650-06e7-458c-a542-9f8815e7303f', '_collection_name': 'healthcare_collection'}, page_content='Disease: Malaria Description: An infectious disease caused by protozoan parasites from the Plasmodium family that can be transmitted by the bite of the Anopheles mosquito or by a contaminated needle or transfusion. Falciparum malaria is the most deadly type. Immediate Action: Consult nearest hospital Medical Advice: avoid oily food Dietary Guidance: avoid non veg